<a href="https://www.kaggle.com/code/amirparvardi/metalagent-v0-1?scriptVersionId=207515555" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## MetalAgent: Metal Music Recommender System

MetalAgent is a Kaggle notebook integrated with GitHub, designed to act as a personalized recommendation system for metal music enthusiasts. Using LangGraph and LangChain, MetalAgent provides curated recommendations, explores subgenres, and engages in interactive conversations about recent releases



## Get set up

Start by installing and importing the LangGraph SDK and LangChain support for the Gemini API.

In [1]:
%pip install -qU 'langgraph==0.2.45' 'langchain-google-genai==2.0.4'

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 


You do not neeed to restart the kernel even if you get the error `ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.`

### Set up your API key

The `GOOGLE_API_KEY` environment variable can be set to automatically configure the underlying API. This works for both the official Gemini Python SDK and for LangChain/LangGraph. 

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [2]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and also** *enable* it.

## Key concepts

LangGraph applications are built around a **graph** structure. As the developer, you define an application graph that models the state transitions for your application. Your app will define a **state** schema, and an instance of that schema is propagated through the graph.

Each **node** in the graph represents an action or step that can be taken. Nodes will make changes to the state in some way through code that you define. These changes can be the result of invoking an LLM, by calling an API, or executing any logic that the node defines.

Each **edge** in the graph represents a transition between states, defining the flow of the program. Edge transitions can be fixed, for example if you define a text-only chatbot where output is always displayed to a user, you may always transition from `chatbot -> user`. The transitions can also be conditional, allowing you to add branching (like an `if-else` statement) or looping (like `for` or `while` loops).

LangGraph is highly extensible and provides a number of features that are not part of this tutorial, such as memory, persistance and streaming. To better understand the key concepts and philophies behind LangGraph, check out their [Conceptual guides](https://langchain-ai.github.io/langgraph/concepts/) and [High-level overview](https://langchain-ai.github.io/langgraph/concepts/high_level/).

## Define core instructions

State is a fundamental concept for a LangGraph app. A state object is passed between every node and transition in the app. Here you define a state object, `RequestState`, that holds the conversation history, a structured request, and a flag indicating if the metalhead has finished placing their request. For simplicity, the "structure" in this request is just a list of strings, but this can be expanded to any Python data structure.

In Python, the LangGraph state object is a Python [dictionary](https://docs.python.org/3/library/stdtypes.html#dict). You can provide a schema for this dictionary by defining it as a [`TypedDict`](https://docs.python.org/3/library/typing.html#typing.TypedDict).

Here you also define the system instruction that the Gemini model will use. You can capture tone and style here, as well as the playbook under which the chatbot should operate.

In [3]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class RequestState(TypedDict):
    """State representing the metalhead's request conversation."""

    # The chat conversation. This preserves the conversation history
    # between nodes. The `add_messages` annotation indicates to LangGraph
    # that state is updated by appending returned messages, not replacing
    # them.
    messages: Annotated[list, add_messages]

    # The metalhead's in-progress request.
    request: list[str]

    # Flag indicating that the order is placed and completed.
    finished: bool


# The system instruction defines how the MetalAgent is expected to behave and includes
# rules for when to call different functions, as well as rules for the conversation, such
# as tone and what is permitted for discussion.
METALAGENT_SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    "You are a MetalAgent, an interactive metal recommendation system. A metalhead will ask you to talk about the "
    "recent metal releases in your collection and you will answer any questions about the releases (and only about "
    "genres and releases - no off-topic discussion, but you can chat about the releases and their band's history). "
    "The metalhead will make a recommendation request releases from 1 or more genres from your collection, "
    "which you will structure and send to the recommendation system. "
    "\n\n"
    "Add items to the metalhead's request with add_to_request, and reset the request with clear_request. "
    "To see the contents of the request so far, call get_request (this is shown to you, not the user) "
    "Calling confirm_request will display the requested genres to the user and returns their response to seeing the list."
    "Respond with a list of releases from metalhead's requested genres and recommend "
    "possible subgenres from the GENRE MENU and add them to the request. "
    "After confirming the requested genres (and possibly subgenres) with the metalhead, call confirm_request and then call place_request. "
    "Once place_request has returned, print the list of releases in the requested subgenres, thank the metalhead and say goodbye!",
)

# This is the message with which the system opens the conversation.
WELCOME_MSG = "Welcome to the MetalAgent recommendation system. Type `q` to quit. What metal genre do you have in mind today?"

## Define a single turn chatboot

To illustrate how LangGraph works, the following program defines a chatbot node that will execute a single turn in a chat conversation using the instructions supplied.

Each node in the graph operates on the state object. The state (a Python dictionary) is passed as a parameter into the node (a function) and the new state is returned. This can be restated as pseudo-code, where `state = node(state)`.

Note: For the `chatbot` node, the state is updated by *adding* the new conversation message. The `add_messages` annotation on `RequestState.messages` indicates that messages are *appended* when returned from a node. Typically state is updated by replacement, but this annotation causes `messages` to behave differently.

In [4]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

# Try using different models. The `pro` models perform the best, especially
# with tool-calling. The `flash` models are super fast, and are a good choice
# if you need to use the higher free-tier quota.
# Check out the features and quota differences here: https://ai.google.dev/pricing
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")


def chatbot(state: RequestState) -> RequestState:
    """The chatbot itself. A simple wrapper around the model's own chat interface."""
    message_history = [METALAGENT_SYSINT] + state["messages"]
    return {"messages": [llm.invoke(message_history)]}


# Set up the initial graph based on our state definition.
graph_builder = StateGraph(RequestState)

# Add the chatbot function to the app graph as a node called "chatbot".
graph_builder.add_node("chatbot", chatbot)

# Define the chatbot node as the app entrypoint.
graph_builder.add_edge(START, "chatbot")

chat_graph = graph_builder.compile()

## Add a human node

Instead of repeatedly running the "graph" in a Python loop, you can use LangGraph to loop between nodes.

The `human` node will display the last message from the LLM to the user, and then prompt them for their next input. Here this is done using standard Python `print` and `input` functions.

The `chatbot` node function has also been updated to include the welcome message to start the conversation.

In [5]:
from langchain_core.messages.ai import AIMessage


def human_node(state: RequestState) -> RequestState:
    """Display the last model message to the user, and receive the user's input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    # If it looks like the user is trying to quit, flag the conversation
    # as over.
    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}


def chatbot_with_welcome_msg(state: RequestState) -> RequestState:
    """The chatbot itself. A wrapper around the model's own chat interface."""

    if state["messages"]:
        # If there are messages, continue the conversation with the Gemini model.
        new_output = llm.invoke([METALAGENT_SYSINT] + state["messages"])
    else:
        # If there are no messages, start with the welcome message.
        new_output = AIMessage(content=WELCOME_MSG)

    return state | {"messages": [new_output]}


# Start building a new graph.
graph_builder = StateGraph(RequestState)

# Add the chatbot and human nodes to the app graph.
graph_builder.add_node("chatbot", chatbot_with_welcome_msg)
graph_builder.add_node("human", human_node)

# Start with the chatbot again.
graph_builder.add_edge(START, "chatbot")

# The chatbot will always go to the human next.
graph_builder.add_edge("chatbot", "human");

Before you can run this, note that if you added an edge from `human` back to `chatbot`, the graph will cycle forever as there is no exit condition. One way to break the cycle is to add a check for a human input like `q` or `quit` and use that to break the loop.

In LangGraph, this is achieved with a conditional edge. This is similar to a regular graph transition, except a custom function is called to determine which edge to traverse.

Conditional edge functions take the state as input, and return a string representing the name of the node to which it will transition.

In [6]:
from typing import Literal


def maybe_exit_human_node(state: RequestState) -> Literal["chatbot", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"


graph_builder.add_conditional_edges("human", maybe_exit_human_node)

chat_with_human_graph = graph_builder.compile()

## Add a "genre" menu

MetalAgent currently has no awareness of the available genres or releases in metal, so it will hallucinate a genre menu. One option would be to hard-code a genre menu into the system prompt. This would work well, but to simulate a system where the genre menu is more dynamic and could respond to actual metal releases, you will put the menu into a custom tool.

There are two types of tools that this system will use. Stateless tools that can be run automatically, and stateful tools that modify the request. The "get current menu" tool is stateless, in that it does not make any changes to the live request, so it can be called automatically.

In a LangGraph app, you can annotate Python functions as tools by applying the `@tools` annotation.


In [7]:
from langchain_core.tools import tool

# Define the path to subgenres and recent releases
Subgenres_path = '/kaggle/input/metal-subgenres/subgenres.txt'
with open(Subgenres_path, 'r', encoding='utf-8') as file:
        subgenres_text = file.read()

NOV_08_2024_path = '/kaggle/input/week-of-november-08-2024/albums_Nov02-Nov08.txt'
with open(NOV_08_2024_path, 'r', encoding='utf-8') as file:
        NOV_08_2024_releases = file.read()
    

@tool
def get_menu() -> str:
    """Provide the latest up-to-date genre menu and recent releases."""
    # Note that this is just hard-coded text, but you could connect this to a live releases
    # database, or you could use Gemini's multi-modal capabilities and take live photos of
    # your favourite concert's band-list or the songs in a playlist and assmble them into an input.

    return f"""
    GENRE MENU: {subgenres_text}
    
    Recent Releases:
    Week of November 08, 2024: {NOV_08_2024_releases}
    """

Now add the new tool to the graph. The `get_menu` tool is wrapped in a [`ToolNode`](https://langchain-ai.github.io/langgraph/reference/prebuilt/#toolnode) that handles calling the tool and passing the response as a message through the graph. The tools are also bound to the `llm` object so that the underlying model knows they exist. As you now have a different `llm` object to invoke, you need to update the `chatbot` node so that it is aware of the tools.

In [8]:
from langgraph.prebuilt import ToolNode


# Define the tools and create a "tools" node.
tools = [get_menu]
tool_node = ToolNode(tools)

# Attach the tools to the model so that it knows what it can call.
llm_with_tools = llm.bind_tools(tools)


def maybe_route_to_tools(state: RequestState) -> Literal["tools", "human"]:
    """Route between human or tool nodes, depending if a tool call is made."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    # Only route based on the last message.
    msg = msgs[-1]

    # When the chatbot returns tool_calls, route to the "tools" node.
    if hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        return "tools"
    else:
        return "human"


def chatbot_with_tools(state: RequestState) -> RequestState:
    """The chatbot with tools. A simple wrapper around the model's own chat interface."""
    defaults = {"request": [], "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([METALAGENT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    # Set up some defaults if not already set, then pass through the provided state,
    # overriding only the "messages" field.
    return defaults | state | {"messages": [new_output]}


graph_builder = StateGraph(RequestState)

# Add the nodes, including the new tool_node.
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)

# Chatbot may go to tools, or human.
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
# Human may go back to chatbot, or exit.
graph_builder.add_conditional_edges("human", maybe_exit_human_node)

# Tools always route back to chat afterwards.
graph_builder.add_edge("tools", "chatbot")

graph_builder.add_edge(START, "chatbot")
graph_with_menu = graph_builder.compile()

Now run the new graph to see how the model uses the genre menu.

**You must uncomment the line containing `.invoke(...)` to run this step.**

In [9]:
# Image(graph_with_menu.get_graph().draw_mermaid_png())

# Remember that you have not implemented ordering yet, so this will loop forever,
# unless you input `q`, `quit` or one of the other exit terms defined in the
# `human_node`.
# Uncomment this line to execute the graph:
# user_msg = "Recommend ten recent black metal releases in diverse subgenres."
# state = graph_with_menu.invoke({"messages": []})

# Things to try:
# - I'd love an espresso drink, what have you got?
# - What teas do you have?
# - Can you do a long black? (this is on the menu as an "Americano" - see if it can figure it out)
# - 'q' to exit.


# pprint(state)

**NOTE**: To save this Kaggle notebook, you will need to comment out the line containing `.invoke(...)` in order for the notebook to Run All cells and finally be able to save your version.